# Pip install

In [52]:
!pip install langchain_community
!pip install faiss-cpu
!pip install -q rank-bm25
!pip install langchain-experimental
!pip -q install pypdf langchain-text-splitters

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# Seeds

In [53]:
import torch 
import numpy as np
import random
import os

seed=824

os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.manual_seed(seed)


In [54]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Imports 

In [55]:
import pandas as pd
import transformers
from transformers import pipeline
import json 
import faiss
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer, CrossEncoder

from langchain_experimental.text_splitter import SemanticChunker
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.retrievers import BaseRetriever
from langchain_core.documents import Document
from tqdm import tqdm 


import re 


# ЕСЛИ ЕСТЬ СТРУКТУРА НАДО ПОЛЬЗОВАТЬСЯ!

# Тут будет расписаны все возможные варианты действий взависимости от данных:


Если есть структура: 


Если нету структуры:



In [56]:
def enrich_structure_heuristic(d: Document) -> dict:
    # Простейшие эвристики заголовков: Markdown, нумерация, ALL-CAPS короткая строка
    text = d.page_content
    heading = None; level = None
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    if lines:
        hline = lines[0]
        if re.match(r"^#{1,6}\s+\S", hline):
            level = len(hline) - len(hline.lstrip("#"))
            heading = hline.lstrip("#").strip()
        elif re.match(r"^\d+(\.\d+){0,3}\s+\S", hline):
            level = 1 + hline.count(".")
            heading = hline
        elif len(hline) < 80 and hline.isupper():
            level = 1; heading = hline
    return {"type": "text", "heading": heading, "heading_level": level}


In [57]:
import re
from dataclasses import dataclass
from typing import List, Dict, Optional
from langchain_core.documents import Document

# ——— эвристика для строки-заголовка ———
_HEADING_MD = re.compile(r"^(#{1,6})\s+(.+)")
_HEADING_NUM = re.compile(r"^(\d+(?:\.\d+){0,5})\s+(.+)")
def is_mostly_upper(s: str, min_len=3, ratio=0.75):
    letters = [c for c in s if c.isalpha()]
    return len(letters) >= min_len and sum(c.isupper() for c in letters)/len(letters) >= ratio

def detect_heading_line(line: str) -> Optional[dict]:
    line = line.strip()
    if not line: return None
    m = _HEADING_MD.match(line)
    if m:
        level = len(m.group(1))
        text = m.group(2).strip()
        return {"heading": text, "level": level, "score": 0.95, "kind":"md"}
    m = _HEADING_NUM.match(line)
    if m:
        depth = m.group(1).count(".") + 1
        text = m.group(2).strip()
        return {"heading": f"{m.group(1)} {text}", "level": min(6, depth), "score": 0.9, "kind":"num"}
    if len(line) < 80 and is_mostly_upper(line):
        return {"heading": line.title(), "level": 1, "score": 0.7, "kind":"caps"}
    if line.endswith(":") and len(line) < 120:
        return {"heading": line[:-1].strip(), "level": 2, "score": 0.55, "kind":"colon"}
    return None

@dataclass
class OutlineNode:
    level: int
    title: str
    page: Optional[int]
    path: List[str]   # хлебные крошки
    node_id: str

def build_outline_from_corpus(corpus: List[Document]) -> Dict[str, List[OutlineNode]]:
    """
    Строим набор заголовков по doc_id (по страницам). Предполагаем, что corpus — постраничные Documents.
    """
    outlines: Dict[str, List[OutlineNode]] = {}
    stacks: Dict[str, List[OutlineNode]] = {}
    counters: Dict[str, int] = {}

    for d in corpus:
        doc_id = d.metadata.get("doc_id","_")
        page = d.metadata.get("page") or d.metadata.get("page_number")
        text = d.page_content or ""
        lines = [ln for ln in (text.splitlines()) if ln.strip()]

        if doc_id not in stacks:
            stacks[doc_id] = []
            outlines[doc_id] = []
            counters[doc_id] = 0

        for ln in lines:
            h = detect_heading_line(ln)
            if not h: 
                continue
            level = h["level"]
            title = h["heading"]
            # срежем стек до parent уровня
            while stacks[doc_id] and stacks[doc_id][-1].level >= level:
                stacks[doc_id].pop()
            path = [n.title for n in stacks[doc_id]] + [title]
            counters[doc_id] += 1
            nid = f"{doc_id}::h{counters[doc_id]}"
            node = OutlineNode(level=level, title=title, page=page, path=path, node_id=nid)
            outlines[doc_id].append(node)
            stacks[doc_id].append(node)

    return outlines


In [58]:
def attach_structure_to_chunks(chunks: List[Document], outlines: Dict[str, List[OutlineNode]]) -> List[Document]:
    """
    Для каждого чанка найдём "ближайший выше" заголовок (по странице/порядку)
    и добавим: heading, heading_level, section_path, parent_id.
    """
    by_doc: Dict[str, List[OutlineNode]] = outlines
    for d in chunks:
        doc_id = d.metadata.get("doc_id","_")
        page = d.metadata.get("page") or d.metadata.get("page_number") or -1
        cand = None
        if doc_id in by_doc:
            # берём последний заголовок на странице <= page, если нет — предыдущих страниц
            cands = [n for n in by_doc[doc_id] if (n.page is None or page is None or int(n.page) <= int(page))]
            if cands:
                cand = cands[-1]
        if cand:
            d.metadata["heading"] = cand.title
            d.metadata["heading_level"] = cand.level
            d.metadata["section_path"] = cand.path
            d.metadata["parent_id"] = cand.node_id
        else:
            # нет явного заголовка — пустые поля
            d.metadata.setdefault("heading", None)
            d.metadata.setdefault("heading_level", None)
            d.metadata.setdefault("section_path", [])
            d.metadata.setdefault("parent_id", None)
    return chunks


In [59]:
_TABLE_HINTS = re.compile(r"\b(table|таблица|колонк|column|row|строк|столб|csv|tsv)\b", re.I)
def is_tabular_like(txt: str) -> bool:
    lines = [ln for ln in txt.splitlines() if ln.strip()]
    if not lines: return False
    bars = sum(ln.count("|") for ln in lines)
    tabs = sum(ln.count("\t") for ln in lines)
    commas = sum(ln.count(",") for ln in lines[:10])
    headerish = any(len(ln.split()) <= len(ln.split(","))+1 for ln in lines[:3])
    hint = bool(_TABLE_HINTS.search(txt))
    return (bars >= 3) or (tabs >= 3) or (hint and headerish) or (commas >= 6)

def label_block_type(d: Document):
    txt = d.page_content[:2000]
    if is_tabular_like(txt):
        d.metadata["type"] = "table"
    elif txt.strip().lower().startswith(("figure", "рис.", "рисунок", "рис ")):
        d.metadata["type"] = "figure"
    else:
        d.metadata["type"] = "text"
    return d


In [60]:
from collections import Counter

def structure_report(chunks: List[Document]):
    lvls = Counter(d.metadata.get("heading_level") for d in chunks)
    types = Counter(d.metadata.get("type") for d in chunks)
    paths_nonempty = sum(1 for d in chunks if d.metadata.get("section_path"))
    print("Heading levels:", dict(lvls))
    print("Types:", dict(types))
    print("Sections with path:", paths_nonempty, "/", len(chunks))

def print_outline_sample(outlines: Dict[str, List[OutlineNode]], k=20):
    for doc_id, nodes in outlines.items():
        print(f"== {doc_id} ==")
        for n in nodes[:k]:
            indent = "  "*(n.level-1)
            print(f"{indent}- p.{n.page} L{n.level}: {n.title}")
        break  # один документ для примера


In [61]:
#Надо сделать 

In [62]:
def composite_text(d: Document) -> str:
    path = " / ".join(d.metadata.get("section_path", [])) if d.metadata.get("section_path") else ""
    head = d.metadata.get("heading") or ""
    typ  = d.metadata.get("type") or "text"
    base = d.page_content
    summ = d.metadata.get("_summary_text")
    head_block = []
    head_block.append(f"[TYPE] {typ}")
    if path: head_block.append(f"[PATH] {path}")
    if head: head_block.append(f"[HEAD] {head}")
    if summ: head_block.append(f"[SUMMARY] {summ}")
    return "\n".join(head_block + ["[TEXT] " + base])

def _tok_with_heading(d: Document) -> List[str]:
    base = _tok(d.page_content)
    head = _tok(d.metadata.get("heading","") or "")
    path = _tok(" / ".join(d.metadata.get("section_path", [])) if d.metadata.get("section_path") else "")
    summ = _tok(d.metadata.get("_summary_text","") or "")
    return base + head*3 + path*2 + summ*2

def dedup_by_chunk_id(docs: List[Document]) -> List[Document]:
    seen, out = set(), []
    for d in docs:
        cid = d.metadata.get("_chunk_id")
        if cid in seen: 
            continue
        seen.add(cid); out.append(d)
    return out

# Cleaning data

In [63]:
import re
import unicodedata
import html

# precompiled bits
_WS_IN_LINE = re.compile(r'[ \t]+')
_URL_RE = re.compile(r'https?://\S+|www\.\S+', flags=re.IGNORECASE)
_EMAIL_RE = re.compile(r'\b[\w\.-]+@[\w\.-]+\.\w+\b')

def cleaning(
    text: str,
    *,
    lower: bool = False,
    keep_newlines: bool = True,
    remove_urls: bool = False,
    remove_emails: bool = False
) -> str:
    """
    Light text normalization (safe defaults).
    - Unicode normalize (NFKC), unescape HTML entities
    - Drop zero-width & control chars (keeps \\n, \\t)
    - Normalize curly quotes/dashes/ellipsis
    - Collapse excess whitespace; optionally preserve newlines
    - (Optional) strip URLs/emails
    - (Optional) lowercase
    """
    if text is None:
        return ""

    s = html.unescape(str(text))
    s = unicodedata.normalize("NFKC", s)

    # remove zero-width & most control chars (keep newlines/tabs)
    s = "".join(
        ch for ch in s
        if not (
            unicodedata.category(ch) in {"Cf", "Cc"} and ch not in ("\n", "\t")
        )
    )

    # typographic normalization
    s = s.translate({
        ord("“"): '"', ord("”"): '"', ord("„"): '"', ord("‟"): '"',
        ord("’"): "'", ord("‘"): "'", ord("‚"): "'", ord("′"): "'",
        ord("–"): "-", ord("—"): "-", ord("−"): "-", ord("‐"): "-",
        ord("…"): "...",
    })

    # optional removals
    if remove_urls:
        s = _URL_RE.sub(" ", s)
    if remove_emails:
        s = _EMAIL_RE.sub(" ", s)

    # whitespace normalization
    s = s.replace("\r\n", "\n").replace("\r", "\n")
    if keep_newlines:
        # collapse spaces/tabs within lines
        s = "\n".join(_WS_IN_LINE.sub(" ", line).strip() for line in s.split("\n"))
        # collapse 3+ newlines -> 2
        s = re.sub(r"\n{3,}", "\n\n", s)
    else:
        s = re.sub(r"\s+", " ", s).strip()

    # tidy spaces before punctuation
    s = re.sub(r"\s+([,.;:!?])", r"\1", s)

    if lower:
        s = s.lower()

    return s

# Loading data

In [64]:
from pathlib import Path
from typing import List
from langchain_community.document_loaders import PyPDFLoader, TextLoader
from langchain_core.documents import Document

def load_any(path: str) -> List[Document]:
    p = Path(path)
    suf = p.suffix.lower()
    if suf == ".pdf":
        docs = PyPDFLoader(str(p)).load()
    elif suf in {".txt", ".md"}:
        docs = TextLoader(str(p), encoding="utf-8").load()
    else:
        raise ValueError(f"Unsupported format: {suf}")

    # normalize + enrich metadata
    out = []
    for i, d in enumerate(docs):
        cleaned = cleaning(d.page_content, remove_urls=True, remove_emails=True)
        meta = {**(d.metadata or {}), "source": str(p), "doc_id": str(p.stem)}
        out.append(Document(page_content=cleaned, metadata=meta))
    return out

# Пример: собрать корпус
# corpus = []
# for p in ["report.pdf", "notes.txt"]:
#     corpus += load_any(p)


In [65]:
#corpus = []
#for p in ["report.pdf", "notes.txt"]:
#    corpus += load_any(p)

## PDF 

## txt 

## anything else?

## general cleaning

# Chunking

## recursive 

In [66]:
recursive_text_splitter=RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=120,
    separators=["\n\n", "\n", " ", ""])

In [67]:
#texts = text_splitter.split_documents(documents)
#chunks = recursive_text_splitter.split_documents(corpus)
#for i, d in enumerate(chunks):
#    d.metadata["_chunk_id"] = i  # критично для фьюжна/логов


## semantic 

In [68]:
hf = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

semantic_text_splitter = SemanticChunker(hf,
                                breakpoint_threshold_type='percentile',
                                breakpoint_threshold_amount=90) # chose which embeddings and breakpoint type and threshold to use


#ПОНЯТЬ 

In [69]:
#chunks_sem = semantic_text_splitter.split_documents(corpus)
#for i, d in enumerate(chunks_sem):
#    d.metadata["_chunk_id"] = i

# Retrival 

In [70]:
'''
Retrieval
1)	BM25 + FIASS
2)	Ансамбль энкодеров
3)	Двуступенчатый ретривал + использование cross-encoder (next sentence prediction) 	
4)	Дополнительная мета, как в задании на кинопоиск
5)	Maximal Marginal Relevance (есть в langchain) помогает разнообразную инфу добавлять в контекст
6)	Hierarchical retrieval + родительские куски текста
7)	Adaptive K - выбираем порог по уверенности а не по количеству 
1)	Multi-hop/ Self-ask - разбивает запрос на части + Self-RAG + ReAct
2)	Chain-of-Thought prompting
'''

'\nRetrieval\n1)\tBM25 + FIASS\n2)\tАнсамбль энкодеров\n3)\tДвуступенчатый ретривал + использование cross-encoder (next sentence prediction) \t\n4)\tДополнительная мета, как в задании на кинопоиск\n5)\tMaximal Marginal Relevance (есть в langchain) помогает разнообразную инфу добавлять в контекст\n6)\tHierarchical retrieval + родительские куски текста\n7)\tAdaptive K - выбираем порог по уверенности а не по количеству \n1)\tMulti-hop/ Self-ask - разбивает запрос на части + Self-RAG + ReAct\n2)\tChain-of-Thought prompting\n'

# Making candidates 

## bm25

In [71]:
#4)	Лематизация ДЛЯ BM25

In [72]:
try:
    import spacy
    nlp = spacy.load("en_core_web_sm")  # для RU: ru_core_news_sm
except Exception:
    nlp = None

def _tok(s: str):
    if nlp:
        doc = nlp(s)
        return [t.lemma_.lower() for t in doc if not t.is_space and not t.is_punct]
    return re.findall(r"\b\w+\b", s.lower())

#bm25_corpus_tokens = [_tok(d.page_content) for d in chunks]
#bm25 = BM25Okapi(bm25_corpus_tokens)

## faiss

In [73]:
#5)	IVF / HNSW / PQ (Product Quantization) + использовать батч (fiass gpu)

In [74]:
#embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2") # FIX
#vs = FAISS.from_documents(chunks, embeddings)

In [75]:
#vs = FAISS.from_documents(chunks, hf)

#faiss_path = "faiss_index"
#vs.save_local(faiss_path)

In [76]:
def build_faiss_augmented(chunks: List[Document], emb: HuggingFaceEmbeddings) -> FAISS:
    """
    Индексируем composite_text(d), но оригинал храним в metadata["_raw"].
    """
    aug_docs = []
    for d in chunks:
        aug_text = composite_text(d)
        md = {**d.metadata, "_raw": d.page_content}
        aug_docs.append(Document(page_content=aug_text, metadata=md))
    return FAISS.from_documents(aug_docs, emb)

def to_raw(doc: Document) -> Document:
    """
    Возвращаем документ с исходным текстом из metadata["_raw"], если он есть.
    """
    raw = doc.metadata.get("_raw")
    if raw:
        return Document(page_content=raw, metadata=doc.metadata)
    return doc

def to_raw_many(docs: List[Document]) -> List[Document]:
    return [to_raw(d) for d in docs]


In [77]:
#emb = HuggingFaceEmbeddings(
#    model_name=embeddings_model,
#    encode_kwargs={"normalize_embeddings": True}  # косинус лучше себя ведёт с нормировкой
#)
#vs = build_faiss_augmented(chunks, emb)

### Ансамбль энкодеров

In [78]:
#hf1 = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
#hf2 = HuggingFaceEmbeddings(model_name="sentence-transformers/multi-qa-mpnet-base-dot-v1")
#vs1 = FAISS.from_documents(chunks, hf1)
#vs2 = FAISS.from_documents(chunks, hf2)

def vec_rrf(query, stores, base_docs, k_each=200, C=60):
    rrf = {}
    for store in stores:
        res = store.similarity_search_with_score(query, k=k_each)
        for r, (d, _) in enumerate(res):
            cid = d.metadata["_chunk_id"]
            rrf[cid] = rrf.get(cid, 0.0) + 1.0/(C + r + 1)
    top = sorted(rrf, key=rrf.get, reverse=True)[:50]
    cid2doc = {d.metadata["_chunk_id"]: d for d in base_docs}
    return [cid2doc[i] for i in top]



## fuze them

In [79]:
'''
def rrf_fusion(query: str, vs: FAISS, bm25: BM25Okapi, *,
               bm25_k: int = 200, vec_k: int = 200, final_k: int = 10, C: int = 60):
    # BM25 топы
    q_tokens = _tok(query)
    scores = bm25.get_scores(q_tokens)
    order = np.argsort(scores)[::-1][:bm25_k]

    # FAISS топы
    vec = vs.similarity_search_with_score(query, k=vec_k)

    # индексируем по _chunk_id
    rrf = {}
    # BM25
    for r, idx in enumerate(order):
        chunk_id = chunks[idx].metadata["_chunk_id"]
        rrf[chunk_id] = rrf.get(chunk_id, 0.0) + 1.0 / (C + r + 1)
    # FAISS
    for r, (doc, _) in enumerate(vec):
        cid = doc.metadata.get("_chunk_id")
        if cid is not None:
            rrf[cid] = rrf.get(cid, 0.0) + 1.0 / (C + r + 1)

    top_ids = sorted(rrf, key=rrf.get, reverse=True)[:final_k]
    cid2doc = {d.metadata["_chunk_id"]: d for d in chunks}
    return [cid2doc[i] for i in top_ids]

'''

'\ndef rrf_fusion(query: str, vs: FAISS, bm25: BM25Okapi, *,\n               bm25_k: int = 200, vec_k: int = 200, final_k: int = 10, C: int = 60):\n    # BM25 топы\n    q_tokens = _tok(query)\n    scores = bm25.get_scores(q_tokens)\n    order = np.argsort(scores)[::-1][:bm25_k]\n\n    # FAISS топы\n    vec = vs.similarity_search_with_score(query, k=vec_k)\n\n    # индексируем по _chunk_id\n    rrf = {}\n    # BM25\n    for r, idx in enumerate(order):\n        chunk_id = chunks[idx].metadata["_chunk_id"]\n        rrf[chunk_id] = rrf.get(chunk_id, 0.0) + 1.0 / (C + r + 1)\n    # FAISS\n    for r, (doc, _) in enumerate(vec):\n        cid = doc.metadata.get("_chunk_id")\n        if cid is not None:\n            rrf[cid] = rrf.get(cid, 0.0) + 1.0 / (C + r + 1)\n\n    top_ids = sorted(rrf, key=rrf.get, reverse=True)[:final_k]\n    cid2doc = {d.metadata["_chunk_id"]: d for d in chunks}\n    return [cid2doc[i] for i in top_ids]\n\n'

In [80]:
def rrf_fusion(query: str, vs: FAISS, bm25: BM25Okapi, base_docs: List[Document],
               bm25_k: int = 200, vec_k: int = 200, final_k: int = 10, C: int = 60):
    q_tokens = _tok(query)
    scores = bm25.get_scores(q_tokens)
    order = np.argsort(scores)[::-1][:bm25_k]

    vec = vs.similarity_search_with_score(query, k=vec_k)

    rrf = {}
    for r, idx in enumerate(order):
        chunk_id = base_docs[idx].metadata["_chunk_id"]
        rrf[chunk_id] = rrf.get(chunk_id, 0.0) + 1.0/(C + r + 1)
    for r, (doc, _) in enumerate(vec):
        cid = doc.metadata.get("_chunk_id")
        if cid is not None:
            rrf[cid] = rrf.get(cid, 0.0) + 1.0/(C + r + 1)

    top_ids = sorted(rrf, key=rrf.get, reverse=True)[:final_k]
    cid2doc = {d.metadata["_chunk_id"]: d for d in base_docs}
    return [cid2doc[i] for i in top_ids]


### ParentDocumentRetriever

# Cross encoder

## Обучение кросс энкодера 

In [81]:
'''
На чем?
1)	Одна фраза из чанка, остальное в чанке позитив, любой другой чанк из другого документа негатив
2)	Заголовок - запрос, содержащиеся в абзаце/параграфе этого заголовка - ответ, если есть структура
3)	Через Spacy получить ключевое слово в абзаце, абзац - ответ 
4)	Прпросить ллм задать вопрос
'''

'\nНа чем?\n1)\tОдна фраза из чанка, остальное в чанке позитив, любой другой чанк из другого документа негатив\n2)\tЗаголовок - запрос, содержащиеся в абзаце/параграфе этого заголовка - ответ, если есть структура\n3)\tЧерез Spacy получить ключевое слово в абзаце, абзац - ответ \n4)\tПрпросить ллм задать вопрос\n'

### training cross encoder on Одна фраза из чанка, остальное в чанке позитив, любой другой чанк из другого документа негатив

In [82]:
from sentence_transformers import InputExample
from torch.utils.data import DataLoader

def make_training_pairs_variant1(docs: List[Document], negatives_per_pos=1):
    # 1) "Одна фраза из чанка" → позитив — сам чанк; негатив — из других доков
    import random, itertools
    examples = []
    by_docid = {}
    for d in docs:
        by_docid.setdefault(d.metadata.get("doc_id","_"), []).append(d)

    all_docs = docs[:]
    for d in docs:
        sentences = re.split(r'(?<=[.!?])\s+', d.page_content)
        if not sentences: 
            continue
        q = sentences[0][:200]
        pos = d.page_content
        # негативы из других doc_id
        other = [x for x in all_docs if x.metadata.get("doc_id") != d.metadata.get("doc_id")]
        negs = random.sample(other, k=min(negatives_per_pos, len(other)))
        examples.append(InputExample(texts=[q, pos], label=1.0))
        for n in negs:
            examples.append(InputExample(texts=[q, n.page_content], label=0.0))
    return examples

#train_ex = make_training_pairs_variant1(chunks, negatives_per_pos=2)
#dl = DataLoader(train_ex, shuffle=True, batch_size=16)
#ce = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', num_labels=1)
#ce.fit(train_dataloader=dl, epochs=1, warmup_steps=100, output_path="ce_finetuned")


### training cross encoder on Заголовок - запрос, содержащиеся в абзаце/параграфе этого заголовка - ответ, если есть структура

### training cross encoder on Через Spacy получить ключевое слово в абзаце, абзац - ответ 

## using cross encoder

In [83]:
from typing import Any, List
from pydantic import BaseModel, Field  # pydantic v2
from langchain_core.retrievers import BaseRetriever
from langchain_core.documents import Document
from sentence_transformers import CrossEncoder

#cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2', device='cuda' if torch.cuda.is_available() else 'cpu')



### adaptive k + MMR(?)

In [84]:
# MMR (diversity): сначала — кандидаты
#mmr_docs = vs.max_marginal_relevance_search(query, k=10, fetch_k=60, lambda_mult=0.5)
def retrieve_mmr(vs, query, k=10, fetch_k=60, lambda_mult=0.5):
    return vs.max_marginal_relevance_search(query, k=k, fetch_k=fetch_k, lambda_mult=lambda_mult)


# Adaptive-K: возьми побольше кандидатов → пропусти через cross-encoder → оставь все с score >= τ
def adaptive_k(query, vs, ce, fetch_k=60, tau=0.35):
    cands = vs.similarity_search(query, k=fetch_k)
    scores = ce.predict([[query, d.page_content] for d in cands])
    picked = [d for d, s in sorted(zip(cands, scores), key=lambda x: x[1], reverse=True) if s >= tau]
    return picked or cands[:5]


In [85]:
def retrieve_then_rerank(query: str, base_docs: List[Document], vs: FAISS, bm25: BM25Okapi,
                         ce: CrossEncoder, pre_k: int = 60, final_k: int = 10,
                         mmr_fetch_k: int = 180, mmr_lambda: float = 0.5) -> List[Document]:
    # 1) RRF (BM25 + FAISS)
    cands = rrf_fusion(query, vs, bm25, base_docs=base_docs, final_k=pre_k)

    # 1b) MMR-альтернатива из FAISS → привести к base_docs и дедуп
    try:
        mmr_alt_raw = retrieve_mmr(vs, query, k=min(pre_k, 50),
                                   fetch_k=min(mmr_fetch_k, 3*pre_k), lambda_mult=mmr_lambda)
        mmr_alt = map_to_base(mmr_alt_raw, base_docs)
        cands = dedup_by_chunk_id(cands + mmr_alt)[:pre_k]
    except Exception:
        pass

    # 2) CE-rerank
    pairs  = [[query, d.page_content] for d in cands]
    scores = ce.predict(pairs)
    ranked = [d for d, s in sorted(zip(cands, scores), key=lambda x: float(x[1]), reverse=True)]
    return ranked[:final_k]


# Generation 

In [86]:
'''
1)	Ролевая инструкция (“Ты эксперт…”) + добавить конкретики
2)	Чёткая структура вывода
3)	Контекст всегда подаётся раньше вопроса
4)	Ансамбль промптов + переписать промпт ллм НАЙТИ ПРОМПТЫ 
5)	
6)	Просить оценить свой ответ перед выводом
7)	Few-shot
'''

'\n1)\tРолевая инструкция (“Ты эксперт…”) + добавить конкретики\n2)\tЧёткая структура вывода\n3)\tКонтекст всегда подаётся раньше вопроса\n4)\tАнсамбль промптов + переписать промпт ллм НАЙТИ ПРОМПТЫ \n5)\t\n6)\tПросить оценить свой ответ перед выводом\n7)\tFew-shot\n'

In [87]:
'''
Generation
1)	Разбить важные куски по краям
2)	Суммаризация контекста
3)	Просить указывать источник каждого факта
4)	Self-consistency sampling - посемплить несколько ответов с разным сидом и температурой, разным порядком, выбрать самый часто встречающийся ллмкой
5)	Отдельная LLM проверяет соответствие ответ-контекст

и на финальную модель скоринга тоже ллм
типа даешь запрос даешь ответ и оцениваешь на сколько хорошо раз справился с задачей

'''

'\nGeneration\n1)\tРазбить важные куски по краям\n2)\tСуммаризация контекста\n3)\tПросить указывать источник каждого факта\n4)\tSelf-consistency sampling - посемплить несколько ответов с разным сидом и температурой, разным порядком, выбрать самый часто встречающийся ллмкой\n5)\tОтдельная LLM проверяет соответствие ответ-контекст\n\nи на финальную модель скоринга тоже ллм\nтипа даешь запрос даешь ответ и оцениваешь на сколько хорошо раз справился с задачей\n\n'

In [88]:
'''

def build_prompt(q: str, ctx_docs: List[Document]):
    ctx = "\n\n".join([f"[S{i+1}] {d.page_content}" for i, d in enumerate(ctx_docs)])
    return f"""You are a careful, citation-first QA assistant.
Use ONLY the snippets [S1..S{len(ctx_docs)}]. Cite as [S#] after each sentence.
If not supported, say "I don't know".

CONTEXT:
{ctx}

QUESTION: {q}

Return JSON with keys: answer (string), citations (list of [S#]), confidence (0..1)."""

'''
# HF pipeline (надёжно на CPU/GPU)
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
GEN_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"
tok = AutoTokenizer.from_pretrained(GEN_MODEL, use_fast=True)
mdl = AutoModelForCausalLM.from_pretrained(GEN_MODEL, torch_dtype="auto", device_map="auto").eval()
gen = pipeline("text-generation", model=mdl, tokenizer=tok)

def answer(query, docs):
    prompt, _ = build_prompt(query, docs, tok)
    out = gen(prompt, max_new_tokens=300, temperature=0.2, top_p=0.9)[0]["generated_text"]
    return out


Device set to use cuda:0


In [89]:
import re, json, math, random
from typing import List, Dict, Any
from langchain_core.documents import Document

# --- токены и обрезка ---
def token_len(text: str, tok) -> int:
    return len(tok(text, add_special_tokens=False).input_ids)

def truncate_by_tokens(text: str, tok, max_tokens: int) -> str:
    ids = tok(text, add_special_tokens=False, truncation=True, max_length=max_tokens).input_ids
    return tok.decode(ids, skip_special_tokens=True)

# --- упаковка контекста в бюджет по токенам ---
def pack_context(docs: List[Document], tok, max_ctx_tokens: int = 2500, max_per_snippet: int = 400):
    packed, used = [], 0
    for i, d in enumerate(docs, 1):
        # если есть сжатие — используем его, иначе сырой текст
        body_raw = d.metadata.get("_summary_text") or d.page_content
        body = body_raw.strip()
        body = truncate_by_tokens(body, tok, max_per_snippet)
        tag = f"[S{i}]"
        src = d.metadata.get("source", "?")
        pg  = d.metadata.get("page", d.metadata.get("page_number", d.metadata.get("_page", "?")))
        header = f"{tag} ({src} p.{pg})"
        chunk = f"{header}\n{body}"
        t = token_len(chunk, tok)
        if used + t > max_ctx_tokens:
            break
        packed.append(chunk); used += t
    return "\n\n".join(packed), len(packed)


# --- перестановка: «важные по краям» ---
def reorder_edge(docs: List[Document]) -> List[Document]:
    # если есть score в metadata — предварительно отсортируем
    ranked = sorted(docs, key=lambda d: d.metadata.get("score", 0), reverse=True)
    left, right, out = 0, len(ranked)-1, []
    while left <= right:
        out.append(ranked[left]); left += 1
        if left <= right:
            out.append(ranked[right]); right -= 1
    return out

# --- парс JSON из вывода модели (робастно) ---
def extract_json(text: str) -> Dict[str, Any]:
    # находим последний JSON-блок
    m = list(re.finditer(r"\{[\s\S]*\}", text))
    if not m: return {}
    block = m[-1].group(0)
    try:
        return json.loads(block)
    except Exception:
        # лёгкий фикс запятых/кавычек не делаем агрессивно — лучше упасть в безопасный режим
        return {}


In [90]:
# Базовая ролевая инструкция (варианты стиля)
PROMPT_STYLES = {
    "concise": "Answer concisely, directly, with exact citations after each sentence.",
    "teacher": "Explain clearly as to an advanced student, but keep each sentence short and cite precisely.",
    "lawyer":  "Be rigorous and conservative; include a citation [S#] after any factual claim.",
    "scientist":"Be cautious, quantify uncertainty, and avoid claims not supported by the snippets."
}

# Few-shot примеры (минимальные, citation-first, JSON)
FEW_SHOTS = [
    {
        "ctx": "[S1] The sky appears blue due to Rayleigh scattering.\n[S2] Rayleigh scattering affects shorter wavelengths.",
        "q":   "Why does the sky look blue?",
        "a_json": {"answer": "The sky appears blue because shorter wavelengths are scattered more strongly by air molecules [S1][S2].",
                   "citations": ["S1","S2"], "confidence": 0.84}
    }
]

def render_few_shots():
    parts = []
    for ex in FEW_SHOTS:
        parts.append(
f"""EXAMPLE
CONTEXT:
{ex['ctx']}

QUESTION: {ex['q']}

OUTPUT (JSON):
{json.dumps(ex['a_json'], ensure_ascii=False)}
""")
    return "\n".join(parts)

def build_prompt(q: str, ctx_docs: List[Document], tok,
                 style: str = "concise",
                 use_few_shot: bool = True,
                 max_ctx_tokens: int = 2500,
                 max_per_snippet: int = 400):
    ctx_docs = reorder_edge(ctx_docs)
    ctx_block, used_n = pack_context(ctx_docs, tok, max_ctx_tokens, max_per_snippet)
    style_txt = PROMPT_STYLES.get(style, PROMPT_STYLES["concise"])
    few = render_few_shots() if use_few_shot and FEW_SHOTS else ""

    prompt = f"""You are a careful, citation-first QA assistant.
{style_txt}
Use ONLY the snippets [S1..S{used_n}]. Every factual sentence MUST end with one or more citations like [S#].
If the answer is not fully supported, say "I don't know".
Return your output STRICTLY as a single JSON object wrapped in <JSON>...</JSON> tags. 
Do not include code fences, explanations, or any extra text.

{few}
CONTEXT:
{ctx_block}

QUESTION: {q}

Format exactly:
<JSON>{{"answer": string, "citations": ["S#"...], "confidence": number between 0 and 1}}</JSON>"""
    return prompt, used_n


def llm_generate_text(prompt: str, *, max_new: int = 512, temp: float = 0.0, top_p: float = 0.9) -> str:
    # Предпочтительно — chat template (Qwen-Instruct слушается лучше)
    if hasattr(tok, "apply_chat_template"):
        messages = [
            {"role": "system", "content": "You are a precise assistant that outputs ONLY JSON wrapped in <JSON> tags."},
            {"role": "user", "content": prompt}
        ]
        input_ids = tok.apply_chat_template(messages, return_tensors="pt", add_generation_prompt=True).to(mdl.device)
        # ✅ явная маска
        import torch
        attention_mask = torch.ones_like(input_ids)

        gen_kwargs = {"max_new_tokens": max_new, "do_sample": (temp > 0.0)}
        if temp > 0.0:
            gen_kwargs["temperature"] = temp
            gen_kwargs["top_p"] = top_p

        with torch.no_grad():
            out_ids = mdl.generate(input_ids=input_ids, attention_mask=attention_mask, **gen_kwargs)
        return tok.decode(out_ids[0][input_ids.shape[-1]:], skip_special_tokens=True)

    # Fallback: pipeline, но без temperature при temp=0.0
    kwargs = {"max_new_tokens": max_new, "do_sample": (temp > 0.0), "return_full_text": False}
    if temp > 0.0:
        kwargs["temperature"] = temp
        kwargs["top_p"] = top_p
    return gen(prompt, **kwargs)[0]["generated_text"]


def extract_json(text: str) -> dict:
    import json, re
    # 1) <JSON>{...}</JSON>
    m = re.findall(r"<JSON>\s*(\{[\s\S]*?\})\s*</JSON>", text)
    if m:
        try:
            return json.loads(m[-1])
        except Exception:
            pass
    # 2) ```json ... ```
    m = re.findall(r"```json\s*(\{[\s\S]*?\})\s*```", text, flags=re.IGNORECASE)
    if m:
        try:
            return json.loads(m[-1])
        except Exception:
            pass
    # 3) last {...} block (rough)
    m = list(re.finditer(r"\{[\s\S]*\}", text))
    if m:
        block = m[-1].group(0)
        try:
            return json.loads(block)
        except Exception:
            return {}
    return {}


def generate_one(query: str, docs: List[Document], gen, tok,
                 style="concise", temp=0.0, top_p=0.9, max_new=512, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed)
    prompt, n_ctx = build_prompt(query, docs, tok, style=style)
    out_text = llm_generate_text(prompt, max_new=max_new, temp=temp, top_p=top_p)
    data = extract_json(out_text)
    return data, out_text, n_ctx


def naive_fallback_answer(query: str, docs: List[Document]) -> dict:
    # минимально осмысленный ответ из первого сниппета, чтобы не было пустых строк
    if not docs:
        return {"answer": "I don't know.", "citations": [], "confidence": 0.0}
    body = (docs[0].metadata.get("_summary_text") or docs[0].page_content or "").strip()
    sents = re.split(r'(?<=[.!?])\s+', body)
    ans = " ".join(sents[:2]).strip()
    ans = (ans[:500] + "…") if len(ans) > 500 else ans
    if not ans:
        ans = "I don't know."
    return {"answer": f"{ans} [S1]", "citations": ["S1"], "confidence": 0.35}


In [91]:
def citation_stats(ans_json: Dict[str, Any], n_ctx: int) -> Dict[str, Any]:
    txt = (ans_json or {}).get("answer", "") or ""
    # найдём все [S#]
    refs = re.findall(r"\[S(\d+)\]", txt)
    refs = [int(x) for x in refs if x.isdigit()]
    valid = [r for r in refs if 1 <= r <= n_ctx]
    coverage = 1.0 if txt.strip()=="" else (1.0 if len(valid)>0 else 0.0)  # минимум: есть ли цитаты
    distinct = sorted(set(valid))
    return {
        "has_json": bool(ans_json),
        "has_answer": bool(txt.strip()),
        "refs_total": len(refs),
        "refs_valid": len(valid),
        "refs_distinct": distinct,
        "coverage": coverage,
    }

# Опционально: скор поддержки через CrossEncoder (если есть)
def support_score_ce(query: str, ans_json: Dict[str, Any], docs: List[Document], cross_encoder=None) -> float:
    if not cross_encoder or not ans_json: 
        return 0.0
    txt = ans_json.get("answer", "") or ""
    if not txt.strip():
        return 0.0
    # возьмём процитированные сниппеты, склеим
    cited_ids = set(int(m) for m in re.findall(r"\[S(\d+)\]", txt) if m.isdigit())
    if not cited_ids:
        return 0.0
    ctx_text = "\n".join([docs[i-1].page_content for i in sorted(cited_ids) if 1 <= i <= len(docs)])
    # пара: (вопрос + ответ) vs (объединённый контекст)
    pair = [[f"{query}\n\nANSWER:\n{txt}", ctx_text]]
    sc = float(cross_encoder.predict(pair)[0])  # 0..1-ish
    return sc


In [92]:
def score_candidate(query, docs, ans_json, n_ctx, cross_encoder=None, w_conf=0.3, w_cit=0.3, w_ce=0.4):
    cs = citation_stats(ans_json, n_ctx)
    conf = float(ans_json.get("confidence", 0.0)) if ans_json else 0.0
    ce  = support_score_ce(query, ans_json, docs, cross_encoder) if cross_encoder else 0.0
    # простая агрегация
    s = w_conf*conf + w_cit*(1.0 if cs["refs_valid"]>0 else 0.0) + w_ce*ce
    return s, {"conf": conf, "cit_ok": cs["refs_valid"]>0, "ce": ce, "stats": cs}

def self_consistency_generate(query: str, docs: List[Document], gen, tok, cross_encoder=None,
                              rounds: int = 5, styles=("concise","teacher","lawyer","scientist"),
                              temps=(0.2, 0.35)):
    cands = []
    for r in range(rounds):
        style = random.choice(styles)
        temp  = random.choice(temps)
        seed  = random.randrange(10**9)
        ret = generate_one(query, docs, gen, tok, style=style, temp=temp, seed=seed)

        # Универсальный разбор результата
        if isinstance(ret, tuple):
            if len(ret) >= 3:
                ans_json, raw, n_ctx = ret[0], ret[1], ret[2]
            else:
                raise ValueError(f"generate_one returned tuple of len {len(ret)}")
        elif isinstance(ret, dict):
            ans_json, raw, n_ctx = ret.get("json", {}), ret.get("raw", ""), int(ret.get("n_ctx", 0))
        else:
            # совсем неожиданный случай
            ans_json, raw, n_ctx = {}, str(ret), 0

        score, info = score_candidate(query, docs, ans_json, n_ctx, cross_encoder=cross_encoder)
        cands.append({
            "json": ans_json, "raw": raw, "style": style, "temp": temp, "seed": seed,
            "score": score, "info": info, "n_ctx": n_ctx
        })
    cands.sort(key=lambda x: x["score"], reverse=True)
    best = cands[0] if cands else {"json": {}, "raw": "", "score": 0.0}
    return best, cands



In [93]:
def build_judge_prompt(query: str, ctx_docs: List[Document], ans_json: Dict[str,Any], tok):
    ctx_block, used_n = pack_context(ctx_docs, tok, max_ctx_tokens=1800, max_per_snippet=300)
    ans = json.dumps(ans_json, ensure_ascii=False)
    return f"""You are a strict judge of factual alignment.
Read CONTEXT and the proposed ANSWER (JSON). 
Score alignment from 0.0 to 1.0 where 1.0 = fully supported, 0.0 = unsupported or hallucinated.
Respond with ONLY a JSON: {{"score": number, "reason": string}}.

CONTEXT:
{ctx_block}

ANSWER:
{ans}
"""

def judge_score(query, docs, ans_json, gen, tok):
    jp = build_judge_prompt(query, docs, ans_json, tok)
    # ✅ temp=0.0 -> do_sample=False, без ошибки
    out = llm_generate_text(jp, max_new=200, temp=0.0, top_p=1.0)
    jd = extract_json(out)
    sc = float(jd.get("score", 0.0)) if isinstance(jd, dict) else 0.0
    return sc, jd



In [94]:
def generate_answer(query: str, ctx_docs: List[Document], gen, tok,
                    cross_encoder=None, use_judge=False):
    best, all_cands = self_consistency_generate(
        query, ctx_docs, gen, tok,
        cross_encoder=cross_encoder,
        rounds=3,                    # можно 2–3 для скорости/стабильности
        styles=("concise","teacher"),
        temps=(0.0, 0.2)
    )
    ans_json = best.get("json") or {}

    # --- Fallback if model failed or answer empty ---
    if not isinstance(ans_json, dict) or not ans_json.get("answer"):
        ans_json = naive_fallback_answer(query, ctx_docs)

    judge = {}
    if use_judge and ans_json:
        sc, judge = judge_score(query, ctx_docs, ans_json, gen, tok)
        if "confidence" in ans_json:
            ans_json["confidence"] = float(max(0.0, min(1.0, 0.5*ans_json["confidence"] + 0.5*sc)))

    # Финальная чистка ссылок
    n_ctx = best.get("n_ctx", len(ctx_docs))
    if "citations" in ans_json and isinstance(ans_json["citations"], list):
        ans_json["citations"] = [c for c in ans_json["citations"]
                                 if isinstance(c, str) and re.fullmatch(r"S([1-9]\d*)", c) and 1 <= int(c[1:]) <= n_ctx]
        ans_json["citations"] = sorted(set(ans_json["citations"]), key=lambda x: int(x[1:]))

    return {
        "answer_json": ans_json,
        "best_raw": best.get("raw",""),
        "score_internal": best.get("score", 0.0),
        "meta": {"style": best.get("style"), "temp": best.get("temp"), "seed": best.get("seed")},
        "judge": judge,
        "candidates": all_cands,
    }



### Summarization context

In [95]:
def build_chunk_summary_prompt(text: str, *, target_sentences: int = 4,
                               mode: str = "neutral", query: str | None = None):
    """
    mode: "neutral" — сжать главное; "query" — выделить релевантное запросу
    Возвращаем JSON: {"summary": "...", "bullets": ["..."], "salient_terms": ["..."]}
    """
    if mode == "query" and query:
        return f"""You compress passages for retrieval-augmented QA.

RULES:
- Use ONLY the given PASSAGE. No external knowledge.
- Extract the {target_sentences} most important facts RELEVANT to the QUERY.
- Quote exact numbers/names; avoid speculation.
- Keep neutral and factual.
- Return ONLY JSON: {{"summary": string, "bullets": [string,...], "salient_terms": [string,...]}}

QUERY:
{query}

PASSAGE:
{text}
"""
    else:
        return f"""You compress passages for retrieval-augmented QA.

RULES:
- Use ONLY the given PASSAGE. No external knowledge.
- Extract the {target_sentences} most important facts.
- Quote exact numbers/names; avoid speculation.
- Keep neutral and factual.
- Return ONLY JSON: {{"summary": string, "bullets": [string,...], "salient_terms": [string,...]}}

PASSAGE:
{text}
"""


In [96]:
from typing import List, Dict, Any

def summarize_docs(docs: List[Document], gen, tok, *,
                   mode: str = "neutral", query: str | None = None,
                   max_input_tokens: int = 1500,      # обрежем слишком длинные входы
                   target_summary_tokens: int = 160,   # ~ длина сжатия
                   target_sentences: int = 4,
                   batch_size: int = 6,
                   only_if_longer_than: int = 500      # сжимать только длинные чанки (по токенам)
                   ) -> List[Document]:
    """
    Генерит сжатия и кладёт их в d.metadata["_summary_text"], d.metadata["_summary_tokens"].
    Ничего не затирает — исходный текст остаётся в page_content.
    """
    # 1) подготовим промпты
    prompts, ix = [], []
    for i, d in enumerate(docs):
        tlen = token_len(d.page_content, tok)
        if tlen < only_if_longer_than:
            # короткие оставляем как есть
            continue
        passage = truncate_by_tokens(d.page_content, tok, max_input_tokens)
        p = build_chunk_summary_prompt(passage, target_sentences=target_sentences,
                                       mode=("query" if (mode=="query" and query) else "neutral"),
                                       query=query)
        prompts.append(p); ix.append(i)

    # 2) батчевой прогон
    outputs: List[str] = []
    for b in range(0, len(prompts), batch_size):
        batch = prompts[b:b+batch_size]
        if not batch: break
        #gen_out = gen(batch, max_new_tokens=220, temperature=0.1, top_p=0.9)
        gen_out = gen(batch, max_new_tokens=220, temperature=0.1, top_p=0.9)
        for item in gen_out:
            item0 = item[0] if isinstance(item, list) else item
            outputs.append(item0["generated_text"])
        # HF pipeline вернёт список; достаём тексты по порядку
        

    # 3) распарсим JSON, соберём финальное сжатие и положим в metadata
    for j, raw in enumerate(outputs):
        i_doc = ix[j]
        d = docs[i_doc]
        jd = extract_json(raw)
        # fallback, если модель не вернула JSON
        summary = (jd.get("summary") if isinstance(jd, dict) else "") or ""
        bullets = (jd.get("bullets") if isinstance(jd, dict) else None)
        if isinstance(bullets, list):
            bullets = [str(x).strip() for x in bullets if str(x).strip()]
        else:
            bullets = []

        # сборка компактного текста-сжатия
        #  - короткий summary
        #  - 3–5 буллетов
        #  - без фантазий: всё только из PASSAGE
        summ_text = summary.strip()
        if bullets:
            bl = "\n".join(f"• {b}" for b in bullets[:6])
            summ_text = (summ_text + ("\n" if summ_text else "") + bl).strip()

        # ограничим итог по токенам
        if summ_text:
            summ_text = truncate_by_tokens(summ_text, tok, target_summary_tokens)
            d.metadata["_summary_text"] = summ_text
            d.metadata["_summary_tokens"] = token_len(summ_text, tok)
        else:
            # если не получилось — не пишем summary, будет использован оригинал
            pass

    return docs


# Using everything 

In [97]:
def map_to_base(docs_vs: List[Document], base_docs: List[Document]) -> List[Document]:
    pool = {d.metadata.get("_chunk_id"): d for d in base_docs}
    out = []
    for d in docs_vs:
        cid = d.metadata.get("_chunk_id")
        if cid in pool:
            out.append(pool[cid])
    return dedup_by_chunk_id(out)

In [98]:
# ===== CE training pairs (вариант 1) =====
from sentence_transformers import InputExample, CrossEncoder
from torch.utils.data import DataLoader

def make_training_pairs_variant1(docs: List[Document], negatives_per_pos=1):
    import random
    examples = []
    all_docs = docs[:]
    for d in docs:
        sentences = re.split(r'(?<=[.!?])\s+', d.page_content)
        if not sentences:
            continue
        q = sentences[0][:200]
        pos = d.page_content
        other = [x for x in all_docs if x.metadata.get("doc_id") != d.metadata.get("doc_id")]
        negs = random.sample(other, k=min(negatives_per_pos, len(other))) if other else []
        examples.append(InputExample(texts=[q, pos], label=1.0))
        for n in negs:
            examples.append(InputExample(texts=[q, n.page_content], label=0.0))
    return examples


# ===== CONFIG-DRIVEN ORCHESTRATOR (patched) =====
from dataclasses import dataclass, asdict
from typing import List, Dict, Any, Optional
from pathlib import Path
import time
import numpy as np

@dataclass
class RAGConfig:
    files: List[str]
    query: str

    # Структура/чанкинг
    use_outline: bool = True
    use_semantic_chunking: bool = False
    chunk_size: int = 800
    chunk_overlap: int = 120

    # Индексы/ретривал
    use_bm25: bool = True
    use_faiss: bool = True
    use_rrf: bool = True
    use_mmr: bool = True
    pre_k: int = 60
    final_k: int = 10

    # Сжатие
    use_summarize: bool = True
    summarize_min_tokens: int = 500

    # Cross-Encoder: инференс
    use_ce: bool = True
    ce_model: str = "cross-encoder/ms-marco-MiniLM-L-6-v2"
    adaptive_tau: Optional[float] = None  # порог для CE, если задан — включается адаптивный отбор

    # Cross-Encoder: тренировка (необязательно)
    train_ce: bool = False
    ce_train_output_path: str = "ce_finetuned"
    ce_train_epochs: int = 1
    ce_train_batch_size: int = 16
    ce_train_warmup_steps: int = 100
    ce_negatives_per_pos: int = 2

    # Эмбеддинги
    embeddings_model: str = "sentence-transformers/all-MiniLM-L6-v2"
    normalize_embeddings: bool = True

    # Генерация/судья
    use_judge: bool = True

def _log(msg: str):
    print(f"[{time.strftime('%H:%M:%S')}] {msg}")

def _rank_with_ce(query: str, cands: List[Document], ce: CrossEncoder, final_k: int, adaptive_tau: Optional[float]):
    """Единый шаблон ранжирования с CE + опциональный adaptive_tau."""
    if not cands:
        return []
    pairs  = [[query, d.page_content] for d in cands]
    scores = ce.predict(pairs, batch_size=64)
    ranked_pairs = sorted(zip(cands, map(float, scores)), key=lambda x: x[1], reverse=True)
    if adaptive_tau is not None:
        picked = [(d, s) for d, s in ranked_pairs if s >= adaptive_tau]
        chosen = picked if picked else ranked_pairs[:final_k]
    else:
        chosen = ranked_pairs[:final_k]
    return [d for d, _ in chosen]

def run_with_config(cfg: RAGConfig) -> Dict[str, Any]:
    _log("Start RAG run")
    _log("Config: " + json.dumps({k:v for k,v in asdict(cfg).items() if k not in {'query'}}, ensure_ascii=False))

    # === 1) Load corpus ===
    _log("Loading corpus...")
    corpus: List[Document] = []
    for p in cfg.files:
        if not Path(p).exists():
            raise FileNotFoundError(f"File not found: {p}")
        corpus += load_any(p)
    assert corpus, "Corpus is empty — проверь пути к файлам."

    # === 2) Chunk ===
    _log(f"Chunking: {'Semantic' if cfg.use_semantic_chunking else 'Recursive'}")
    if cfg.use_semantic_chunking:
        _emb_tmp = HuggingFaceEmbeddings(model_name=cfg.embeddings_model)
        semantic_text_splitter = SemanticChunker(_emb_tmp,
                           breakpoint_threshold_type='percentile',
                           breakpoint_threshold_amount=90)
        chunks: List[Document] = semantic_text_splitter.split_documents(corpus)
    else:
        rc = RecursiveCharacterTextSplitter(
            chunk_size=cfg.chunk_size, chunk_overlap=cfg.chunk_overlap,
            separators=["\n\n", "\n", " ", ""]
        )
        chunks: List[Document] = rc.split_documents(corpus)
    for i, d in enumerate(chunks):
        d.metadata["_chunk_id"] = i

    # === 3) Outline & structure ===
    if cfg.use_outline:
        _log("Building outline & attaching structure...")
        outlines = build_outline_from_corpus(corpus)
        chunks = attach_structure_to_chunks(chunks, outlines)
        chunks = [label_block_type(d) for d in chunks]
        structure_report(chunks)
    else:
        _log("Outline skipped")
        chunks = [label_block_type(d) for d in chunks]

    # === 4) Dedup ===
    _log("Dedup chunks by text...")
    seen, uniq = set(), []
    for d in chunks:
        key = re.sub(r"\s+", " ", d.page_content.strip().lower())[:2000]
        if key in seen:
            continue
        seen.add(key); uniq.append(d)
    chunks = uniq
    _log(f"Chunks after dedup: {len(chunks)}")

    # === 5) Build indices ===
    bm25_inst = None
    vs = None
    if cfg.use_bm25:
        _log("Building BM25...")
        bm25_inst = BM25Okapi([_tok_with_heading(d) for d in chunks])
    if cfg.use_faiss:
        _log("Building FAISS...")
        emb = HuggingFaceEmbeddings(
            model_name=cfg.embeddings_model,
            encode_kwargs={"normalize_embeddings": cfg.normalize_embeddings}
        )
        vs = build_faiss_augmented(chunks, emb) 

    # === 6) (Опционально) Query-aware summarization для широкой витрины ===
    if cfg.use_summarize and (cfg.use_bm25 or cfg.use_faiss):
        _log("Wide candidate set for query-aware summarization...")
        if cfg.use_rrf and (bm25_inst is not None) and (vs is not None):
            wide = rrf_fusion(cfg.query, vs, bm25_inst, base_docs=chunks, final_k=cfg.pre_k)
        elif bm25_inst is not None:
            q_tokens = _tok(cfg.query)
            scores = bm25_inst.get_scores(q_tokens)
            order = np.argsort(scores)[::-1][:cfg.pre_k]
            wide = [chunks[i] for i in order]
        else:
            wide_raw = vs.similarity_search(cfg.query, k=cfg.pre_k)
            wide = map_to_base(wide_raw, chunks)
        summarize_docs(wide, gen, tok, mode="query", query=cfg.query, only_if_longer_than=cfg.summarize_min_tokens)
        _log("Summarization done")
        #if cfg.use_faiss:
        #    _log("Rebuilding FAISS to include summaries...")
        #    emb = HuggingFaceEmbeddings(
        #    model_name=cfg.embeddings_model,
        #    encode_kwargs={"normalize_embeddings": cfg.normalize_embeddings}
        #    )
        #    vs = build_faiss_augmented(chunks, emb)
    else:
        _log("Summarization skipped")

    # === 7) Cross-Encoder (optional training) ===
    ce_infer = None
    if cfg.use_ce:
        if cfg.train_ce:
            _log("Training CrossEncoder on-the-fly...")
            train_ex = make_training_pairs_variant1(chunks, negatives_per_pos=cfg.ce_negatives_per_pos)
            dl = DataLoader(train_ex, shuffle=True, batch_size=cfg.ce_train_batch_size)
            ce_train = CrossEncoder(cfg.ce_model, num_labels=1, max_length=512)
            ce_train.fit(
                train_dataloader=dl,
                epochs=cfg.ce_train_epochs,
                warmup_steps=cfg.ce_train_warmup_steps,
                output_path=cfg.ce_train_output_path
            )
            _log(f"CE saved to: {cfg.ce_train_output_path}")
            ce_infer = CrossEncoder(cfg.ce_train_output_path, device='cuda' if torch.cuda.is_available() else 'cpu', max_length=512)
        else:
            ce_infer = CrossEncoder(cfg.ce_model, device='cuda' if torch.cuda.is_available() else 'cpu', max_length=512)

    # === 8) Retrieval (final context) ===
    _log("Retrieval for final context...")
    ctx_docs: List[Document] = []
    if cfg.use_ce:
        # RRF → CE
        if cfg.use_rrf and (bm25_inst is not None) and (vs is not None):
            cands = rrf_fusion(cfg.query, vs, bm25_inst, base_docs=chunks, final_k=cfg.pre_k)
            # + MMR-альтернатива из FAISS (RAW) и дедуп
            if cfg.use_mmr and vs is not None:
                try:
                    mmr_alt_raw = retrieve_mmr(vs, cfg.query, k=min(cfg.pre_k, 50),
                           fetch_k=min(3*cfg.pre_k, 180), lambda_mult=0.5)
                    mmr_alt = map_to_base(mmr_alt_raw, chunks)
                    cands   = dedup_by_chunk_id(cands + mmr_alt)[:cfg.pre_k]
                except Exception:
                    pass
            ctx_docs = _rank_with_ce(cfg.query, cands, ce_infer, cfg.final_k, cfg.adaptive_tau)

        # Только BM25 → CE
        elif (bm25_inst is not None) and (vs is None):
            q_tokens = _tok(cfg.query)
            scores = bm25_inst.get_scores(q_tokens)
            order = np.argsort(scores)[::-1][:cfg.pre_k]
            cands = [chunks[i] for i in order]
            ctx_docs = _rank_with_ce(cfg.query, cands, ce_infer, cfg.final_k, cfg.adaptive_tau)

        # Только FAISS → CE (обязательно to_raw)
        elif (vs is not None) and (bm25_inst is None):
            cands_raw = vs.similarity_search(cfg.query, k=cfg.pre_k)
            cands = map_to_base(cands_raw, chunks)
            ctx_docs = _rank_with_ce(cfg.query, cands, ce_infer, cfg.final_k, cfg.adaptive_tau)

        else:
            raise RuntimeError("Нет индексов для ретривала (включи BM25 и/или FAISS).")

    else:
        # Без CE: RRF или одиночный индекс (+опц. MMR)
        if cfg.use_rrf and (bm25_inst is not None) and (vs is not None):
            cands = rrf_fusion(cfg.query, vs, bm25_inst, base_docs=chunks, final_k=max(cfg.pre_k, cfg.final_k))
            if cfg.use_mmr and vs is not None:
                try:
                    mmr_alt_raw = retrieve_mmr(vs, cfg.query, k=min(cfg.pre_k, 50), fetch_k=min(3*cfg.pre_k, 180))
                    mmr_alt     = map_to_base(mmr_alt_raw, chunks)
                    cands       = dedup_by_chunk_id(cands + mmr_alt)[:cfg.pre_k]
                except Exception:
                    pass
        elif bm25_inst is not None:
            q_tokens = _tok(cfg.query)
            scores = bm25_inst.get_scores(q_tokens)
            order = np.argsort(scores)[::-1][:cfg.pre_k]
            cands = [chunks[i] for i in order]
        else:
            cands = to_raw_many(vs.similarity_search(cfg.query, k=cfg.pre_k))
        if cfg.adaptive_tau is not None:
            _log("adaptive_tau задан, но CE отключён — возьмём обычный top-K.")
        ctx_docs = cands[:cfg.final_k]

    _log(f"Context docs: {len(ctx_docs)}")

    # === 9) Generation ===
    _log("Generation...")
    result = generate_answer(cfg.query, ctx_docs, gen, tok,
                             cross_encoder=(ce_infer if cfg.use_ce else None),
                             use_judge=cfg.use_judge)

    # === 10) Diagnostics ===
    _log("Done.")
    diag = {
        "files": cfg.files,
        "query": cfg.query,
        "ctx_ids": [d.metadata.get("_chunk_id") for d in ctx_docs],
        "ctx_heads": [d.metadata.get("heading") for d in ctx_docs],
        "ctx_paths": [d.metadata.get("section_path") for d in ctx_docs],
        "types": [d.metadata.get("type") for d in ctx_docs],
        "config": asdict(cfg)
    }
    return {"result": result, "context_docs": ctx_docs, "diagnostics": diag}


In [99]:
import json, re, unicodedata

def _slugify(s: str) -> str:
    s = s.lower()
    s = re.sub(r"[^a-z0-9/_\-\s]", "", s)
    s = s.replace(" ", "_")
    s = re.sub(r"_+", "_", s).strip("_")
    return s

def _section_id_from_doc(d: Document) -> str:
    # приоритет: section_path → heading → doc_id
    path = d.metadata.get("section_path") or []
    if isinstance(path, list) and path:
        return _slugify("/".join(path))
    head = d.metadata.get("heading")
    if head:
        return _slugify(head)
    return _slugify(d.metadata.get("doc_id", "book"))

def _page_str_from_doc(d: Document, page_shift: int = 1) -> str:
    # PyPDFLoader обычно даёт 0-based page; в датасете могут быть «нативные» номера
    for key in ("true_page","page","page_number","_page"):
        if key in d.metadata and d.metadata[key] is not None:
            try:
                p = int(d.metadata[key])
                return str(p + page_shift)
            except Exception:
                return str(d.metadata[key])
    return "?"

def build_references(ans_json: dict, ordered_docs: List[Document]) -> dict:
    # берём только процитированные сниппеты
    cited_ids = []
    for c in (ans_json or {}).get("citations", []):
        m = re.fullmatch(r"S(\d+)", c.strip())
        if m:
            cited_ids.append(int(m.group(1)))
    cited_ids = sorted(set(i for i in cited_ids if 1 <= i <= len(ordered_docs)))

    if not cited_ids:
        # fallback: тогда считаем процитированными топ-3
        cited_ids = list(range(1, min(3, len(ordered_docs)) + 1))

    sections, pages = [], []
    for i in cited_ids:
        d = ordered_docs[i-1]
        sections.append(_section_id_from_doc(d))
        pages.append(_page_str_from_doc(d))
    # уникальность с сохранением порядка
    def _uniq_keep(seq): 
        seen=set(); out=[]
        for x in seq:
            if x not in seen:
                seen.add(x); out.append(x)
        return out
    return {"sections": _uniq_keep(sections), "pages": _uniq_keep(pages)}

def build_context_for_submission(ordered_docs: List[Document], max_chars: int = 8000) -> str:
    # оставим те же [S#], это помогает валидации и читаемости
    parts = []
    used = 0
    for i, d in enumerate(ordered_docs, 1):
        block = f"[S{i}] {d.page_content.strip()}"
        if used + len(block) + 2 > max_chars:
            break
        parts.append(block); used += len(block) + 2
    return "\n\n".join(parts)

def finalize_answer_text(ans_json: dict) -> str:
    # из JSON берём «answer», fallback — пусто
    return (ans_json or {}).get("answer", "") or ""


In [100]:
# Если gen/tok ещё не определены:
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
# GEN_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"
# tok = AutoTokenizer.from_pretrained(GEN_MODEL, use_fast=True)
# mdl = AutoModelForCausalLM.from_pretrained(GEN_MODEL, torch_dtype="auto", device_map="auto").eval()
# gen = pipeline("text-generation", model=mdl, tokenizer=tok)

CFG = RAGConfig(
    files=["report.pdf", "notes.txt"],
    query="Кратко: как устроена модель доходов в отчёте и чем она отличается от прошлой версии?",

    use_outline=True,
    use_semantic_chunking=False,
    use_bm25=True,
    use_faiss=True,
    use_rrf=True,
    use_mmr=True,
    pre_k=60,
    final_k=10,

    use_summarize=True,
    summarize_min_tokens=500,

    use_ce=True,
    ce_model="cross-encoder/ms-marco-MiniLM-L-6-v2",
    adaptive_tau=None,          # например, 0.35 для порогового отбора

    # онлайновое обучение CE (по желанию)
    train_ce=False,             # True — обучит CE на текущих чанках
    ce_train_output_path="ce_finetuned",
    ce_train_epochs=1,
    ce_train_batch_size=16,
    ce_train_warmup_steps=100,
    ce_negatives_per_pos=2,

    embeddings_model="sentence-transformers/all-MiniLM-L6-v2",
    normalize_embeddings=True,
    use_judge=True
)

#run = run_with_config(CFG)
#print(json.dumps(run["result"]["answer_json"], ensure_ascii=False, indent=2))
#print("\nDiagnostics:", json.dumps(run["diagnostics"], ensure_ascii=False, indent=2))



In [101]:
# ===== CACHED STATE =====
from dataclasses import dataclass
from typing import Optional, List, Dict, Any

@dataclass
class RAGState:
    chunks: List[Document]
    bm25: Optional[BM25Okapi]
    vs: Optional[FAISS]
    ce: Optional[CrossEncoder]
    cfg_base: RAGConfig  # базовый конфиг без query

def prepare_state(files: List[str], cfg_base: RAGConfig) -> RAGState:
    _log("Preparing state (ONE TIME)...")
    # 1) Load corpus once
    corpus: List[Document] = []
    for p in files:
        if not Path(p).exists():
            raise FileNotFoundError(f"File not found: {p}")
        corpus += load_any(p)
    assert corpus, "Corpus is empty."

    # 2) Chunk once
    if cfg_base.use_semantic_chunking:
        _emb_tmp = HuggingFaceEmbeddings(model_name=cfg_base.embeddings_model)
        semantic_text_splitter = SemanticChunker(_emb_tmp,
                           breakpoint_threshold_type='percentile',
                           breakpoint_threshold_amount=90)
        chunks: List[Document] = semantic_text_splitter.split_documents(corpus)
    else:
        rc = RecursiveCharacterTextSplitter(
            chunk_size=cfg_base.chunk_size, chunk_overlap=cfg_base.chunk_overlap,
            separators=["\n\n", "\n", " ", ""]
        )
        chunks: List[Document] = rc.split_documents(corpus)
    for i, d in enumerate(chunks):
        d.metadata["_chunk_id"] = i

    # 3) Outline/structure once
    if cfg_base.use_outline:
        outlines = build_outline_from_corpus(corpus)
        chunks = attach_structure_to_chunks(chunks, outlines)
        chunks = [label_block_type(d) for d in chunks]
        structure_report(chunks)
    else:
        chunks = [label_block_type(d) for d in chunks]

    # 4) Dedup once
    seen, uniq = set(), []
    for d in chunks:
        key = re.sub(r"\s+", " ", d.page_content.strip().lower())[:2000]
        if key in seen: 
            continue
        seen.add(key); uniq.append(d)
    chunks = uniq
    _log(f"Chunks after dedup: {len(chunks)}")

    # 5) Build indices once
    bm25_inst = None
    vs = None
    if cfg_base.use_bm25:
        _log("Building BM25 (ONE TIME)...")
        bm25_inst = BM25Okapi([_tok_with_heading(d) for d in chunks])
    if cfg_base.use_faiss:
        _log("Building FAISS (ONE TIME)...")
        emb = HuggingFaceEmbeddings(
            model_name=cfg_base.embeddings_model,
            encode_kwargs={"normalize_embeddings": cfg_base.normalize_embeddings}
        )
        # ВНИМАНИЕ: НЕ ПЕРЕСТРАИВАЕМ ПОСЛЕ САММАРИ, индекс по сырому/композитному тексту строим один раз
        vs = build_faiss_augmented(chunks, emb)

    # 6) Cross-Encoder once
    ce_infer = None
    if cfg_base.use_ce:
        ce_infer = CrossEncoder(cfg_base.ce_model, device='cuda' if torch.cuda.is_available() else 'cpu', max_length=512)

    _log("State ready.")
    return RAGState(chunks=chunks, bm25=bm25_inst, vs=vs, ce=ce_infer, cfg_base=cfg_base)

# === Retrieval per-query using prepared state ===
def retrieve_context_for_query(query: str, state: RAGState, pre_k: int, final_k: int, use_rrf=True, use_mmr=True):
    chunks, bm25_inst, vs, ce = state.chunks, state.bm25, state.vs, state.ce

    # Широкая витрина (без перестройки индексов!)
    if use_rrf and (bm25_inst is not None) and (vs is not None):
        cands = rrf_fusion(query, vs, bm25_inst, base_docs=chunks, final_k=pre_k)
        if use_mmr and vs is not None:
            try:
                mmr_alt = retrieve_mmr(vs, query, k=min(pre_k, 50), fetch_k=min(3*pre_k, 180), lambda_mult=0.5)
                mmr_alt = map_to_base(mmr_alt, chunks)   # ВНИМАНИЕ: map_to_base, без mmr_alt_raw
                cands = dedup_by_chunk_id(cands + mmr_alt)[:pre_k]
            except Exception:
                pass
    elif bm25_inst is not None:
        q_tokens = _tok(query)
        scores = bm25_inst.get_scores(q_tokens)
        order = np.argsort(scores)[::-1][:pre_k]
        cands = [chunks[i] for i in order]
    elif vs is not None:
        cands_raw = vs.similarity_search(query, k=pre_k)
        cands = map_to_base(cands_raw, chunks)
    else:
        raise RuntimeError("No indices available.")

    # CE rerank (если есть)
    if ce is not None:
        pairs  = [[query, d.page_content] for d in cands]
        scores = ce.predict(pairs, batch_size=64)
        ranked = [d for d,_ in sorted(zip(cands, map(float, scores)), key=lambda x: x[1], reverse=True)]
        ctx_docs = ranked[:final_k]
    else:
        ctx_docs = cands[:final_k]
    return ctx_docs

def run_for_query_with_state(question: str, state: RAGState,
                             pre_k: int = 60, final_k: int = 10,
                             summarize_min_tokens: int = 500,
                             use_judge: bool = True,
                             self_consistency_rounds: int = 2):
    _log("Retrieval...")
    ctx_docs = retrieve_context_for_query(question, state, pre_k=pre_k, final_k=final_k,
                                          use_rrf=state.cfg_base.use_rrf, use_mmr=state.cfg_base.use_mmr)
    # Query-aware summarization ТОЛЬКО для упаковки (НЕ перестраиваем FAISS!)
    summarize_docs(ctx_docs, gen, tok, mode="query", query=question,
                   only_if_longer_than=summarize_min_tokens)

    _log("Generation...")
    # Вариант ускорения: меньше раундов self-consistency
    best, all_cands = self_consistency_generate(
        question, ctx_docs, gen, tok,
        cross_encoder=(state.ce if state.cfg_base.use_ce else None),
        rounds=self_consistency_rounds,
        styles=("concise","teacher")  # можно 2 стиля для скорости
    )
    ans_json = best["json"] or {}
    judge = {}
    if use_judge and ans_json:
        sc, judge = judge_score(question, ctx_docs, ans_json, gen, tok)
        if "confidence" in ans_json:
            ans_json["confidence"] = float(max(0.0, min(1.0, 0.5*ans_json["confidence"] + 0.5*sc)))

    # Ссылки на секции/страницы
    sections = []
    pages    = []
    for d in ctx_docs:
        sp = d.metadata.get("section_path") or []
        if sp: sections.append("/".join(sp))
        pg = d.metadata.get("page") or d.metadata.get("page_number") or d.metadata.get("_page")
        if pg is not None: pages.append(str(pg))
    refs_json = json.dumps({"sections": sorted(set(sections))[:6], "pages": sorted(set(pages))[:10]}, ensure_ascii=False)

    # Контекст для submission (склеим разумно)
    ctx_text = "\n\n".join(
        f"[S{i+1}] {(d.metadata.get('_summary_text') or d.page_content).strip()}"
        for i, d in enumerate(ctx_docs)
    )[:12000]  # чтобы не раздуть CSV

    return {
        "answer_json": ans_json,
        "context": ctx_text,
        "references": refs_json,
        "ctx_docs": ctx_docs,
        "judge": judge
    }

# === FAST SUBMISSION ===
def make_submission_fast(queries_json_path: str, files: List[str], out_csv: str,
                         cfg_base: Optional[RAGConfig] = None):
    # базовый конфиг без query
    cfg_base = cfg_base or RAGConfig(
        files=files, query="",
        use_outline=True,
        use_semantic_chunking=False,
        use_bm25=True, use_faiss=True, use_rrf=True, use_mmr=True,
        pre_k=60, final_k=10,
        use_summarize=True, summarize_min_tokens=500,
        use_ce=True, ce_model="cross-encoder/ms-marco-MiniLM-L-6-v2",
        adaptive_tau=None,
        train_ce=False,
        embeddings_model="sentence-transformers/all-MiniLM-L6-v2",
        normalize_embeddings=True,
        use_judge=True
    )

    # 1) Считываем вопросы
    data = json.loads(Path(queries_json_path).read_text(encoding="utf-8"))
    items = data["queries"] if isinstance(data, dict) and "queries" in data else data
    assert isinstance(items, list) and items, "queries.json must be a list or have a 'queries' list"

    # 2) Готовим состояние один раз
    state = prepare_state(files, cfg_base)

    # 3) Пробегаем вопросы
    rows = []
    for it in tqdm(items, desc="Answering (fast)"):
        qid = it.get("id") or it.get("query_id") or it.get("ID") or it.get("qid") or len(rows)
        qtext = it.get("question") or it.get("query") or it.get("text") or ""
        qid_int = int("".join(ch for ch in str(qid) if ch.isdigit()) or 0)

        out = run_for_query_with_state(
            qtext, state,
            pre_k=cfg_base.pre_k, final_k=cfg_base.final_k,
            summarize_min_tokens=cfg_base.summarize_min_tokens,
            use_judge=cfg_base.use_judge,
            self_consistency_rounds=2  # ускоряем
        )
        ans = out["answer_json"].get("answer", "")
        # На всякий случай чистим переносы в answer, чтобы CSV был аккуратнее
        ans = ans.replace("\r", " ").replace("\n", " ").strip()

        rows.append({
            "ID": qid_int,
            "context": out["context"],
            "answer": ans,
            "references": out["references"]
        })

    df = pd.DataFrame(rows, columns=["ID", "context", "answer", "references"])
    df.to_csv(out_csv, index=False)
    return df


In [102]:
import pandas as pd
import json
from pathlib import Path

def run_for_query(files, question: str) -> dict:
    cfg = RAGConfig(
        files=files,
        query=question,
        use_outline=True,
        use_semantic_chunking=False,
        use_bm25=True,
        use_faiss=True,
        use_rrf=True,
        use_mmr=True,
        pre_k=60,
        final_k=10,
        use_summarize=True,
        summarize_min_tokens=500,
        use_ce=True,
        ce_model="cross-encoder/ms-marco-MiniLM-L-6-v2",
        adaptive_tau=None,
        train_ce=False,
        embeddings_model="sentence-transformers/all-MiniLM-L6-v2",
        normalize_embeddings=True,
        use_judge=True
    )
    out = run_with_config(cfg)
    res = out["result"]
    ordered = res.get("ordered_docs", out.get("context_docs", []))  # на всякий
    ctx = build_context_for_submission(ordered)
    refs = build_references(res.get("answer_json", {}), ordered)
    ans = finalize_answer_text(res.get("answer_json", {}))
    return {"context": ctx, "answer": ans, "references": json.dumps(refs, ensure_ascii=False)}

def make_submission(queries_json_path: str, files: list, out_csv: str = "submission.csv"):
    data = json.loads(Path(queries_json_path).read_text(encoding="utf-8"))

    # Унифицированое извлечение списка запросов
    if isinstance(data, dict):
        items = data.get("queries") or data.get("data") or data.get("items") or data.get("questions") or []
    elif isinstance(data, list):
        items = data
    else:
        raise ValueError("Unsupported queries.json format")

    rows = []
    for it in tqdm(items, desc="Answering"):
        qid = it.get("ID") or it.get("query_id") or it.get("id")
        question = it.get("question") or it.get("query") or it.get("text")
        if qid is None or not question:
            continue
        # безопасно к int (обычно query_id числовой; если строка-число — тоже ок)
        try:
            qid_int = int(qid)
        except Exception:
            qid_int = int("".join(ch for ch in str(qid) if ch.isdigit()) or 0)

        r = run_for_query(files, question)
        rows.append({
            "ID": qid_int,
            "context": r["context"],
            "answer": r["answer"],
            "references": r["references"]
        })

    df = pd.DataFrame(rows).sort_values("ID")
    df.to_csv(out_csv, index=False)
    return df

# пример вызова:
files = ["/kaggle/input/casml-generative-ai-hackathon/Dataset_RAG (1)/book.pdf"]

df_sub = make_submission_fast(
    "/kaggle/input/casml-generative-ai-hackathon/Dataset_RAG (1)/queries.json",
    files,
    "submission.csv"
)
display(df_sub.head())


[19:40:35] Preparing state (ONE TIME)...
Heading levels: {None: 1, 1: 2274, 2: 1386}
Types: {'text': 1284, 'table': 2317, 'figure': 60}
Sections with path: 3660 / 3661
[19:40:52] Chunks after dedup: 3661
[19:40:52] Building BM25 (ONE TIME)...
[19:42:49] Building FAISS (ONE TIME)...
[19:43:00] State ready.


Answering (fast):   0%|          | 0/50 [00:00<?, ?it/s]

[19:43:00] Retrieval...
[19:43:00] Generation...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Answering (fast):   2%|▏         | 1/50 [00:11<09:07, 11.17s/it]

[19:43:11] Retrieval...
[19:43:11] Generation...


...

[19:51:59] Retrieval...
[19:52:00] Generation...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Answering (fast):  98%|█████████▊| 49/50 [09:10<00:12, 12.22s/it]

[19:52:11] Retrieval...
[19:52:11] Generation...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Answering (fast): 100%|██████████| 50/50 [09:21<00:00, 11.23s/it]


,ID,context,answer,references
0,1,[S1] explores questions like these. Psychology...,The scientific method in psychology involves d...,"{""sections"": [""2 • Critical Thinking Questions..."
1,2,"[S1] the other hand, serve as interconnected i...","A neuron consists of several different parts, ...","{""sections"": [""102 3 • Summary"", ""3 • Review Q..."
2,3,[S1] modification of work by Ryan Vaarsi)\nNRE...,The stages of sleep include: 1. **Stage 1**: ...,"{""sections"": [""118 4 • States of Consciousness..."
3,4,[S1] Operant conditioning is based on the work...,Operant conditioning is a form of learning whe...,"{""sections"": [""16 • Summary 629"", ""192 6 • Lea..."
4,5,[S1] sequence for individuals around the world...,Problem-solving in psychology involves identif...,"{""sections"": [""214 7 • Thinking and Intelligen..."


In [103]:
df_sub['answer']

0     The scientific method in psychology involves d...
1     A neuron consists of several different parts, ...
2     The stages of sleep include:  1. **Stage 1**: ...
3     Operant conditioning is a form of learning whe...
4     Problem-solving in psychology involves identif...
5     According to the Atkinson-Shiffrin model, ther...
6     Key components of emotion include physiologica...
7     The major personality traits in the Five Facto...
8     Social psychology is the study of how people i...
9     The sociocultural model in therapy considers t...
10    The history of psychology began in the late 18...
11    Wilhelm Wundt was a German scientist who found...
12    Functionalism is a psychological perspective t...
13    Freud's contributions to psychology include de...
14    Gestalt principles are rules or guidelines tha...
15    Classical conditioning is a form of learning i...
16    Skinner's contributions to behaviorism include...
17    Maslow's hierarchy of needs consists of fi